In [ ]:
#| default_exp utils.misc

In [ ]:
#| export
import pprint

In [ ]:
# |export
class nict(dict):
    def __init__(self, d=dict(), /, **kwargs):
        if d is None: d = {}
        else:
            assert isinstance(d, dict), f'expected dict, got {type(d)}'
            d = d.copy()

        d.update(kwargs)
        for k,v  in d.items():
            if isinstance(v, dict):
                d[k] = nict(**v)
            elif isinstance(v, list):
                d[k] = [nict(**item) if isinstance(item, dict) else item for item in v]
            elif isinstance(v, tuple):
                d[k] = tuple(nict(**item) if isinstance(item, dict) else item for item in v)
            else:
                d[k] = v

        super().__init__(**d)

    def __getattr__(self, key):
        return self[key] if key in self else super().__getattribute__(key)

    def __setattr__(self, key, value):
        self[key] = value

    def __delattr__(self, key):
        del self[key]

    def __dir__(self):
        return list(self.keys()) + dir(super())

    def _to_dict(self):
        return {k: v._to_dict() if isinstance(v, nict) else v for k, v in self.items()}

    def _repr_markdown_(self):
        return f'```json\n{pprint.pformat(self._to_dict(), indent=2)}\n```'


In [ ]:
_test_dict = {'a':1, 'b': {'c':1, 'd':2}, 'c': {'c':1, 'd':2}, 'd': {'c':1, 'd':2},
              'e': {'c':1, 'd':2}, 'f': {'c':1, 'd':2, 'e': 4, 'f':[1,2,3,4,5]}}

It can take a dictionary, and optionally the keys you want to override.

In [ ]:
n = nict(_test_dict, a=2, b={'c':2, 'd':3})
n

```json
{ 'a': 2,
  'b': {'c': 2, 'd': 3},
  'c': {'c': 1, 'd': 2},
  'd': {'c': 1, 'd': 2},
  'e': {'c': 1, 'd': 2},
  'f': {'c': 1, 'd': 2, 'e': 4, 'f': [1, 2, 3, 4, 5]}}
```

Or you can expand a dictionary into it

In [ ]:
n = nict(**_test_dict)
n

```json
{ 'a': 1,
  'b': {'c': 1, 'd': 2},
  'c': {'c': 1, 'd': 2},
  'd': {'c': 1, 'd': 2},
  'e': {'c': 1, 'd': 2},
  'f': {'c': 1, 'd': 2, 'e': 4, 'f': [1, 2, 3, 4, 5]}}
```

If for some reason you want to convert it back to a plain dict:

In [ ]:
type(n._to_dict()) # This converts recursively

dict

And of course the whole point is to be able to access elements with dot notation.

In [ ]:
n.a

1

In [ ]:
n.f.e

4